In [ ]:
# Import libraries
from pathlib import Path
import acoular as ac
import numpy as np
import time
initialTime = time.time()

#---------------------------------------------------------------------------------------------------------------
# initial definitions
# define microphone geometry
micgeofile = Path().resolve() / "in/array_position_data/bassoon_cage_64_optimized.xml"

# define calculation grid
resolution = 0.05#0.02

# define audio data
name = 'in/array_audio_data/2025-01-28_15-59-01_400437.h5'

# define time related variables
start = 25#83#26        # in s
stop = 35#35#92#36
framerate = 0.125           # in f/s

# define freqency resolution
bandwith = 3            # Octave = 1; 1/3 Octave = 3
frequency_bands = [1000,2000] # Array with mid frequencies of frequency bands

#---------------------------------------------------------------------------------------------------------------
# calculation of fixed values
data = ac.MaskedTimeSamples(name = name)
m = ac.MicGeom(from_file=micgeofile)
g = ac.RectGrid3D(x_min=0.35, x_max=1.15, y_min=0.15, y_max=1.15, z_min=0.25, z_max=1.65, increment=resolution)
st = ac.SteeringVector(grid=g, mics=m, steer_type='true location')
frameLength = 1/framerate

print('init time: ' + str(time.time()-initialTime) + ' s')

# begin evaluation loop
# result = np.zeros()
calcTime = 0
for i, currentFreqBand in enumerate(frequency_bands):

    frame = 0
    for frame in range(0,int((stop-start)*framerate)):
        calcTime = time.time()

        print('Frame length: ' + str((start+(frameLength*(frame+1)))-(start+(frameLength*frame))) + ' s' + '(Frame ' + str(frame)+')')
        print('Frequency Band: ' + str(i) + ' (' + str(currentFreqBand) + ' Hz)')

        data = ac.MaskedTimeSamples(name = name, start = (start+(frameLength*frame))*data.sample_freq, stop = (start+(frameLength*(frame+1)))*data.sample_freq)

        f = ac.PowerSpectra(source=data, window='Hanning', overlap='50%', block_size=512)#256)#128)
        b = ac.BeamformerCleansc(freq_data=f, steer=st)

        result[frame] = b.synthetic(1000, 3)

        print('Frame ' + str(frame) + ' calculated. Calculation time:' + str(time.time()-calcTime))
        frame = frame + 1
    i = i+1

# result_Lp = ac.L_p(result)


init time: 0.03000164031982422 s
Frame length: 8.0 s(Frame 0)
Frequency Band: 0 (1000 Hz)
